Testbed for LSTM

In [1]:
import numpy as NP
import pickle
import matplotlib.pyplot as PLT
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.decoders import LSTMDecoder
import sklearn
from matplotlib import pyplot as PLT
import keras
keras_v1=int(keras.__version__[0])<=1
training_range=[0, 0.5]
valid_range=[0.5,0.65]
testing_range=[0.65, 0.8]
lag=0


File=open("C:/Users/gerri/Coding_Folder/Data_Folder/DS_Final_Data/example_data_hc.pickle",mode='rb')
[neural_data,pos_binned]=pickle.load(File,)
print(f"neural_data \n{NP.shape(neural_data)}"+"\n"+f"pos_binned \n{NP.shape(pos_binned)}")

"""
neural_data:    When neurons Fired. num_bins x num_neurons. 1 when fired, 0 when not.
pos_binned       :    X and Y velocities
"""

#Remove all rows of pos binned and neural data where pos binned is NaN
rmv_time=NP.where(NP.isnan(pos_binned[:,0]) | NP.isnan(pos_binned[:,1]))
X_kf=NP.delete(neural_data,rmv_time,0)
pos_binned=NP.delete(pos_binned,rmv_time,0)

#get velocity of positions binned
temp=NP.diff(pos_binned,axis=0)
vels_binned=NP.concatenate((temp,temp[-1:,:]),axis=0)

#We now determine acceleration, by taking diff between each position.
temp2=NP.diff(vels_binned,axis=0)
acc_binned=NP.concatenate((temp2,temp2[-1:,:]),axis=0)

#Y_kf is the positions binned, velocities binned, then acc_binned in numpy array.
Y_kf=NP.concatenate((pos_binned,vels_binned,acc_binned),axis=1)

#Center Y along axis=0
# Y_mean=NP.mean(Y_kf,axis=0)
# Y_std=NP.std(Y_kf,axis=0)
# Y_kf=(Y_kf-Y_mean)/Y_std

nd_sum=NP.nansum(X_kf,axis=0) #Total number of spikes of each neuron
nd_mean=NP.mean(nd_sum)
nd_std=NP.std(nd_sum)
nd_thresh=nd_mean-nd_std*2


rmv_nrn=NP.where(nd_sum<100) #Find neurons who have less than 100 spikes total
rmv_nrn_std=NP.where(nd_sum<nd_thresh) #Find neurons who have less than 2 std of dataset. Gerrik
print(f"rmv_nrn \n{NP.shape(rmv_nrn)}"+"\n"+f"rmv_nrn_std \n{NP.shape(rmv_nrn_std)}")

#Centered firing covariance.
Firing_Covariate=NP.delete(X_kf,rmv_nrn,1)
Firing_Covariate_G=NP.delete(X_kf,rmv_nrn,1)

#If there is known lag between the two datasets.
num_examples=X_kf.shape[0]
if lag==0:
    print("No lag")
elif lag<0:
    Y_kf=Y_kf[-lag:,:]
    X_kf=X_kf[0:num_examples+lag,:]
elif lag>0:
    Y_kf=Y_kf[0:num_examples-lag,:]
    X_kf=X_kf[lag:num_examples,:]


num_examples_kf=X_kf.shape[0]    


neural_data 
(28039, 58)
pos_binned 
(28039, 2)
rmv_nrn 
(1, 12)
rmv_nrn_std 
(1, 0)
No lag


In [2]:
for IDX,Test_Size in enumerate([0.5,0.7,0.8]):
    X_Train, X_Test, Y_Train, Y_Test =sklearn.model_selection.train_test_split(X_kf,Y_kf,test_size=Test_Size)
    X_Valid, X_Test, Y_Valid, Y_Test =sklearn.model_selection.train_test_split(X_Test,Y_Test,test_size=Test_Size)
    
    #Need to recenter, because Simpson's paradox. The grey doesnt work, but second centering does for some reason.
    # X_kf_train_mean=NP.nanmean(X_Train,axis=0)
    # X_kf_train_std=NP.nanstd(X_Train,axis=0)
    # X_kf_valid_mean=NP.nanmean(X_Train,axis=0)
    # X_kf_valid_std=NP.nanstd(X_Train,axis=0)
    # X_kf_test_mean=NP.nanmean(X_Train,axis=0)
    # X_kf_test_std=NP.nanstd(X_Train,axis=0)
    # X_Train=(X_Train-X_kf_train_mean)/X_kf_train_std
    # X_Test=(X_Test-X_kf_test_mean)/X_kf_test_std
    # X_Valid=(X_Valid-X_kf_valid_mean)/X_kf_valid_std
    
    X_kf_train_mean=NP.nanmean(X_Train,axis=0)
    X_kf_train_std=NP.nanstd(X_Train,axis=0)
    X_kf_train=(X_Train-X_kf_train_mean)/X_kf_train_std
    X_kf_test=(X_Test-X_kf_train_mean)/X_kf_train_std
    X_kf_valid=(X_Valid-X_kf_train_mean)/X_kf_train_std
    
    y_kf_train_mean=NP.mean(Y_Train,axis=0)
    y_kf_train=Y_Train-y_kf_train_mean
    y_kf_test=Y_Test-y_kf_train_mean
    y_kf_valid=Y_Valid-y_kf_train_mean
    
    model_lstm=LSTMDecoder(units=400,dropout=0,num_epochs=5)

    #Fit model
    model_lstm.fit(X_kf_train,y_kf_train)

    #Get predictions
    Y_valid_predicted_lstm=model_lstm.predict(X_kf_valid)

    #Get metric of fit
    R2s_lstm=get_R2(y_kf_valid,Y_valid_predicted_lstm)
    print('R2s:', R2s_lstm)
    
    R2_kf=get_R2(Y_Valid,Y_valid_predicted_lstm)
    print('R2:',R2_kf[0:2]) #I'm just printing the R^2's of the 1st and 2nd entries that correspond to the positions
    #Next I'll get the rho^2 (the pearson correlation squared)
    rho_kf=get_rho(Y_Valid,Y_valid_predicted_lstm)
    print('rho2:',rho_kf[0:2]**2) #I'm just printing the rho^2's of the 1st and 2nd entries that correspond to the positions
    
    PLT.subplot(3,2,IDX+1)
    PLT.plot(Y_Valid[2000:2100,0]+y_kf_train_mean[0],'b',label="Raw")
    PLT.plot(Y_valid_predicted_lstm[2000:2100,0]+y_kf_train_mean[0],'r',label="Predict")
    PLT.title(f"Plot predict vs true. Test size {Test_Size}.")
    PLT.legend()

IndexError: tuple index out of range